# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [29]:
import os
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from tokenization_kobert import KoBertTokenizer
import numpy as np
from sklearn.model_selection import StratifiedKFold
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader

## Set Hyperparameters

In [30]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [31]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [40]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, '../output')

## Cleanlab

In [39]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    evaluation_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

In [40]:

#3-fold 진행
prob_list = []

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
batch_size = 256

for i, (train_index, test_index) in enumerate(skf.split(data, data.target)):
    data_train = BERTDataset(data.iloc[train_index], tokenizer)
    data_eval = BERTDataset(data.iloc[test_index], tokenizer)

    train_dataloader = DataLoader(data_train, batch_size=batch_size)
    eval_dataloader = DataLoader(data_eval, batch_size=batch_size)
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset=data_eval,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    
    # Train the model
    trainer.train()
    
    model.eval()
    
    olist = torch.from_numpy(trainer.predict(data_eval).predictions)
    olist = torch.softmax(olist, dim=1).detach().numpy()
    prob_list.extend(zip(test_index, olist))
    

prob_list.sort()
data["prob"] = [value for idx,value in prob_list]

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


In [ ]:
pred_probs = np.stack(data["prob"])
ordered_label_issues = find_label_issues(labels=data['target'], pred_probs=pred_probs, return_indices_ranked_by='self_confidence')
data["new_label"] = data['target']

print("ordered_label_issues", len(ordered_label_issues))
print(data.columns)

for issue_idx in ordered_label_issues:
    new_label = data.iloc[issue_idx, 5].argmax(-1)
    data.iloc[issue_idx, 2] = new_label

data.to_csv("delete_noise.csv", index=False)